# Neural Network - Length of Stay (LOS)

This ML model predicts days from admission to discharge for the patients in the MIMIC dataset who lived

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

Using TensorFlow backend.


# Data Setup
Creates separate clean dataframes for both sets of patients - lived & died

In [22]:
labs = pd.read_csv('../Resources/labsNew.csv')
labs.head()

,SUBJECT_ID,HADM_ID,Albumin,Anion Gap,Bicarbonate,"Bilirubin, Total",Chloride,"Chloride, Whole Blood",Creatinine,Glucose_Blood_Gas,...,PT,PTT,Phosphate,Platelet Count,Potassium,"Potassium, Whole Blood",Sodium,"Sodium, Whole Blood",Urea Nitrogen,White Blood Cells
0,2,163353.0,0.0,0.0,0.0,9.3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.1
1,3,145834.0,1.8,17.0,25.0,0.8,99.0,114.0,3.2,265.0,...,14.8,125.7,4.8,179.0,5.4,3.7,136.0,139.0,36.0,15.1
2,4,185777.0,2.8,17.0,24.0,2.2,97.0,0.0,0.5,0.0,...,12.3,31.3,3.2,207.0,3.1,0.0,135.0,0.0,9.0,9.7
3,5,178980.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,309.0,0.0,0.0,0.0,0.0,0.0,13.9
4,6,107064.0,2.7,17.0,16.0,0.2,107.0,95.0,3.5,106.0,...,12.5,55.2,4.1,198.0,4.9,4.2,135.0,135.0,86.0,22.7


In [24]:
lived = pd.read_csv('../Resources/admissions_survived.csv')
lived.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,LENGTH_OF_STAY
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1,1.144444
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,5.496528
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1,6.768056
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1,2.856944
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1,3.534028


In [25]:
lived_df = pd.merge(lived, labs, on='HADM_ID')
lived_df.head()

,ROW_ID,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,PT,PTT,Phosphate,Platelet Count,Potassium,"Potassium, Whole Blood",Sodium,"Sodium, Whole Blood",Urea Nitrogen,White Blood Cells
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,...,12.4,30.1,3.7,259.0,4.4,0.0,140.0,0.0,17.0,5.1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,17.5,42.0,3.2,95.0,3.9,3.6,143.0,140.0,14.0,9.4
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,...,11.9,26.6,3.0,216.0,4.2,3.5,140.0,133.0,16.0,10.9
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,...,12.4,22.5,3.8,215.0,4.1,0.0,139.0,0.0,13.0,9.8
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,...,12.8,34.6,4.5,269.0,3.3,0.0,134.0,0.0,50.0,12.2


In [26]:
lived_df.columns

Index(['ROW_ID', 'SUBJECT_ID_x', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'LENGTH_OF_STAY', 'SUBJECT_ID_y', 'Albumin',
       'Anion Gap', 'Bicarbonate', 'Bilirubin, Total', 'Chloride',
       'Chloride, Whole Blood', 'Creatinine', 'Glucose_Blood_Gas',
       'Glucose_Chemistry', 'Hematocrit', 'Hematocrit, Calculated',
       'Hemoglobin_Blood_Gas', 'Hemoglobin_Hematology', 'INR(PT)', 'Lactate',
       'Magnesium', 'PT', 'PTT', 'Phosphate', 'Platelet Count', 'Potassium',
       'Potassium, Whole Blood', 'Sodium', 'Sodium, Whole Blood',
       'Urea Nitrogen', 'White Blood Cells'],
      dtype='object')

In [29]:
lived_df = lived_df.drop(['ROW_ID', 'SUBJECT_ID_x', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DISCHARGE_LOCATION',
                         'RELIGION', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'SUBJECT_ID_y', 'LANGUAGE'], axis = 1)

In [31]:
lived_df_dummy = pd.get_dummies(lived_df)
lived_df_dummy.head()

,LENGTH_OF_STAY,Albumin,Anion Gap,Bicarbonate,"Bilirubin, Total",Chloride,"Chloride, Whole Blood",Creatinine,Glucose_Blood_Gas,Glucose_Chemistry,...,DIAGNOSIS_WOUND INFECTION,DIAGNOSIS_WOUND INFECTION R/O SEPSIS,DIAGNOSIS_WOUND INFECTION;HYPOTENSION,DIAGNOSIS_WOUND INFECTION;TELEMETRY,DIAGNOSIS_WOUND TO RLE,DIAGNOSIS_WOUND ULCER,DIAGNOSIS_ZENKER'S DIVERTICULUM/SDA,DIAGNOSIS_ZONE 2 NECK INJURY,DIAGNOSIS_ZOSTER VS. MENINGITIS,DIAGNOSIS_ZYGOMATIC FRACTURE LEFT
0,1.144444,0.0,13.0,28.0,0.4,103.0,0.0,0.6,0.0,102.0,...,0,0,0,0,0,0,0,0,0,0
1,5.496528,0.0,12.0,23.0,0.0,112.0,0.0,0.7,118.0,128.0,...,0,0,0,0,0,0,0,0,0,0
2,6.768056,4.1,13.0,27.0,0.8,104.0,98.0,0.7,122.0,141.0,...,0,0,0,0,0,0,0,0,0,0
3,2.856944,0.0,14.0,24.0,0.5,105.0,0.0,0.9,0.0,100.0,...,0,0,0,0,0,0,0,0,0,0
4,3.534028,3.2,19.0,23.0,0.4,95.0,0.0,1.6,0.0,378.0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
lived_df_dummy.columns

Index(['LENGTH_OF_STAY', 'Albumin', 'Anion Gap', 'Bicarbonate',
       'Bilirubin, Total', 'Chloride', 'Chloride, Whole Blood', 'Creatinine',
       'Glucose_Blood_Gas', 'Glucose_Chemistry',
       ...
       'DIAGNOSIS_WOUND INFECTION', 'DIAGNOSIS_WOUND INFECTION R/O SEPSIS',
       'DIAGNOSIS_WOUND INFECTION;HYPOTENSION',
       'DIAGNOSIS_WOUND INFECTION;TELEMETRY', 'DIAGNOSIS_WOUND TO RLE',
       'DIAGNOSIS_WOUND ULCER', 'DIAGNOSIS_ZENKER'S DIVERTICULUM/SDA',
       'DIAGNOSIS_ZONE 2 NECK INJURY', 'DIAGNOSIS_ZOSTER VS. MENINGITIS',
       'DIAGNOSIS_ZYGOMATIC FRACTURE LEFT'],
      dtype='object', length=14070)

In [33]:
# assignment and reshaping of LIVED data
data = lived_df_dummy.drop(["LENGTH_OF_STAY"], axis=1)
target = lived_df_dummy["LENGTH_OF_STAY"].values.reshape(-1, 1)

print(f"data shape: {data.shape}")
print(f"target shape: {target.shape}")

data shape: (52294, 14069)
target shape: (52294, 1)


# Length of Stay Model